<img src="../imgs/Adevinta-ULPGC-logo.jpg" width="530px" align="right">

# **Redes neuronales 6**

## **Desvanecimiento del gradiente**

Las redes convolutivas adolecen de **desvanecimiento del gradiente**, causado por la acumulación de capas consecutivas que, progresivamente, reducen el gradiente hasta hacerlo insignificante.

<div align="center">
    <img src="imgs/dg.svg">
</div>

Para recordar cómo se lleva a cabo el entrenamiento sobre una neurona, comenzamos viendo la expresión del error:


$$
E(\vec{w}) =  \sum_j \left( \text{Sig}( \sum_i{w_i  e_i^j} ) - l^j  \right) ^2
$$

Si queremos derivar parcialmente con respecto a alguna $w_k$, haremos:

$$
\frac{\partial{E}}{\partial{w_k}} =  \sum_j \left[  2 \left( \text{Sig}( \sum_i{w_i  e_i^j} ) - l^j  \right) \cdot \text{Sig}( \sum_i{w_i  e_i^j} ) \cdot [1 - \text{Sig}( \sum_i{w_i  e_i^j} )  ] \cdot e_k^j \right]
$$

Si nos fijamos en el factor:

$$
\text{Sig}( \sum_i{w_i  e_i^j} ) \cdot [1 - \text{Sig}( \sum_i{w_i  e_i^j} )]
$$

veremos que si $\sum_i{w_i  e_i^j}$ crece con valores que se vayan alejando progresivamente del cero, el resultado de la expresión anterior tiende muy rápidamente al cero. Veámoslo con ejemplos:

In [1]:
import math

def sigmoide(x):
    return 1/(1+math.exp(-x))

def derivada_sigmoide(x):
    return sigmoide(x)*(1-sigmoide(x))

print("Derivada del sigmoide en 0 = ",derivada_sigmoide(0))
print("Derivada del sigmoide en 1 = ",derivada_sigmoide(1))
print("Derivada del sigmoide en 5 = ",derivada_sigmoide(5))
print("Derivada del sigmoide en 10 = ",derivada_sigmoide(10))
print("Derivada del sigmoide en 30 = ",derivada_sigmoide(30))

Derivada del sigmoide en 0 =  0.25
Derivada del sigmoide en 1 =  0.19661193324148185
Derivada del sigmoide en 5 =  0.006648056670790033
Derivada del sigmoide en 10 =  4.5395807735907655e-05
Derivada del sigmoide en 30 =  9.348077867342945e-14


Si analizamos el caso de una red y quisiéramos calcular, por ejemplo, la derivada parcial de $w^2_1$, tendríamos que la expresión es:

$$
\frac{\partial{E}}{\partial{w^2_1}} =  \sum_j \left[  2 \left( \sigma_1( \Sigma_1 ) - l^j  \right) \cdot \sigma_1( \Sigma_1 ) (1- \sigma_1(\Sigma_1))  \cdot \sigma_2( \Sigma_2 ) (1- \sigma_2(\Sigma_2))  \cdot \text{out}_3    \right]
$$

<div align="center">
    <img src="imgs/dg_multi.svg" width=60%>
</div>

El asunto ahora empeora ya que que tenemos dos factores: $\sigma_1( \Sigma_1 ) (1- \sigma_1(\Sigma_1))  \cdot \sigma_2( \Sigma_2 ) (1- \sigma_2(\Sigma_2))$ en lugar de uno que reducen el gradiente. Esto hace que $w^2_1$ se actualice muy lentamente. Como imaginarás, la cosa va peor a medida que los pesos a actualizar se encuentran en capas más atrás.

## **Conexiones residuales**

Cuando implementamos una red obligamos, por medio del entrenamiento, que esta mapee una cierta entrada $x$, por medio de una función $F(x)$, en unas determinadas salidas $\hat{y}$. Es, por tanto, misión de la red encontrar una función $F(x)$ lo suficientemente acertada. En la figura inferior, la red izquierda es la clásica red que hemos estado viendo, pero la red derecha crea un "enlace" entre la entrada y la salida de la segunda capa. ¿Para qué? ¿Soluciona eso el problema del desvanecimiento del gradiente?

Vemos que las dos capas de red de la izquierda intentan construir $F(x)$, pero en el diseño de la derecha esa conexión directa entre la entrada y la salida obliga a las dos capas de red a que, si quiere tener a la salida la función $F(x)$, debe entonces forzar a que la función $g_2(f(x)) = g_1(f(x)) - f(x)$, para que, después de sumar $f(x)$, la salida sea la $F(x)$ buscada.

<div align="center">
    <img src="imgs/residuals_.svg" width=60%>
</div>

La siguiente función matemática nos va a ayudar a entender la finalidad de esta técnica. Supongamos que tenemos la entrada $x$ a la que se le aplica la función $f$, generando $f(x)$. Ahora el camino del valor $f(x)$ se divide en dos. En primer lugar entra como valor a la función $g$, la cual devuelve $g(f(x))$. Este valor se suma al segundo camino que tomaba $f(x)$, dando como resultado final $g(f(x))+f(x)$.

<div align="center">
    <img src="imgs/cadena_derivada_.svg" width=50%>
</div>

Lo interesante de esto viene cuando realizamos su derivada. Si la calculamos, obtendremos la expresión:

$$
g'(f(x)) \cdot f'(x) + f'(x)
$$

Si interpretamos que $f$ es una primera capa de una red neuronal y $g$ es la siguiente capa, vemos que la conexión residual nos permite tener la derivada de $f$ sin estar multiplicada por la derivada de $g$, lo cual hacía que el gradiente disminuyera sensiblemente.

Esta técnica apareció por primera vez en [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf), pertenciente a Microsoft Research en 2015. Esto dio origen a la red **ResNet**.

<div align="center">
    <img src="imgs/resnet_.png">
</div>